# 01 FrontEnd 코드 작성

###### Home.pug
```pug
doctype html
html(lang="en")
  head
    meta(charset="UTF-8")
    meta(http-equiv="X-UA-Compatible", content="IE=edge")
    meta(name="viewport", content="width=device-width, initial-scale=1.0")
    title Joom
    //- MVP CSS 사용
    link(rel="stylesheet", href="https://unpkg.com/mvp.css@1.12/mvp.css") 
  body 
    header
      h1 Joom 
    main 
      div#welcome
        form 
          input(placeholder="room name", required, type='text')
          button Enter Room
    script(src="/socket.io/socket.io.js")
    script(src="/public/js/app.js") 
```


###### app.js
```javascript
const socket = io();

const welcome = document.querySelector("#welcome");
const form = welcome.querySelector("form");

form.addEventListener("submit", (event) => {
  event.preventDefault();
  const input = form.querySelector("input");
  // websocket.send와 같음
  // 원하는 것을 emit해주면됨
  
  /**
   * event 이름
   * payload 보내고 싶은 데이터
   * 많은 인자들을 보낼수 있음
   * function은 마지막 인자로 
   * function 서버에서 호출할수 있는 함수
   */
  socket.emit("enter_room", { payload: input.value }, () => {
    console.log("server is done!");
  });
  input.value = "";
});


```

---

# 02 BackEnd 코드 작성

###### server.js

```javascript
import express from "express";
import http from "http";
import SocketIO from "socket.io";

const app = express();
const port = 3000;

// json file 입출력 설정
app.use(express.json());

// pug 설정
app.set("view engine", "pug");
app.set("views", __dirname + "/views");

// static 작업
app.use("/public", express.static(__dirname + "/public"));

// route 설정
app.get("/", (_, res) => res.render("home"));
app.get("/*", (_, res) => res.redirect("/"));

// http server
const httpServer = http.createServer(app);

// io server
const wsServer = SocketIO(httpServer);

wsServer.on("connection", (socket) => {
  socket.on("enter_room", (msg, done) => {
    console.log(msg);
    setTimeout(() => {
      // 호출은 backend에서 하지만 실행은 frontend에서 진행됨
      done();
    }, 10000);
  });
});

httpServer.listen(port, () => {
  console.log(`Joom app listening on http://localhost:${port}`);
});


```

---